In [2]:
from gensim.models import Word2Vec
model = Word2Vec.load('model.pkl')

In [3]:
import utils
contexts = utils.load_contexts('../corpora/ad-nouns-contexts-100k', 'горшок')
print(contexts[0])
print(len(contexts))

['час', 'время', 'инцидент', 'хватить', 'некоторый', 'легенда', 'говориться', 'золото', 'находиться', 'конец', 'радуга', 'глубина', '22', 'фут', 'почва', 'становиться']
34188


In [12]:
from collections import Counter
import numpy as np
min_count = 50
all_words = [
    w for w, cnt in Counter(w for ctx in contexts for w in ctx).items()
    if cnt >= min_count]
print(len(all_words))
w2v_vecs = np.array([model[w] for w in all_words if w in model])
print(len(w2v_vecs))

1819
1817


In [13]:
import kmeans
n_senses = 6
km = kmeans.KMeans(w2v_vecs, k=n_senses, metric='cosine')

kmeans: X (85, 300)  centres (6, 300)  delta=0.001  maxiter=10  metric=cosine
kmeans: 5 iterations  cluster sizes: [13 19 12 12 16 13]
kmeans: X (1817, 300)  centres (6, 300)  delta=0.001  maxiter=10  metric=cosine
kmeans: 8 iterations  cluster sizes: [241 256 361 365 161 433]


In [14]:
all_words = np.array(all_words)
for sense in range(n_senses):
    indices = km.Xtocentre == sense
    distances = km.distances[indices]
    sense_words = all_words[indices]
    min_indices = np.argsort(distances)[:10]
    distances[min_indices]
    print(sense, sense_words[min_indices])

0 ['болезнь' 'запускать' 'исчезать' 'куча' 'зимний' 'скворец' 'бить' 'шар'
 'отправлять' 'учитель']
1 ['лена' 'оставлять' 'вроде' 'горшочек' 'тарелка' 'шум' 'расставлять'
 'рождение' 'потолок' 'материал']
2 ['стиль' 'десяток' 'смех' 'ворота' 'скамья' 'инструмент' 'наполнять'
 'важно' 'обед' 'пиво']
3 ['садик' 'кружиться' 'мама' 'убеждаться' 'начальник' 'рыба' 'почва'
 'касаться' 'пакет' 'народ']
4 ['михаил' 'хозяин' 'ложиться' 'волос' 'орудие' 'качество' 'известно'
 'штанишки' 'темнота' 'привыкать']
5 ['спасибо' 'слово' 'угодно' 'земляной' 'увидеть' 'старуха' 'дым' 'стенка'
 'листовой' 'змея']
